[Paper](https://arxiv.org/abs/2306.06031)

[GitHub](https://github.com/AI4Finance-Foundation/FinGPT/tree/master)

[Hugging face](https://huggingface.co/FinGPT)

[Training script](https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Benchmark)

[Inference](https://github.com/AI4Finance-Foundation/FinGPT/blob/master/FinGPT_Inference_Llama2_13B_falcon_7B_for_Beginners.ipynb)

[Install packages permanently in Colab](https://www.google.com/search?q=avoid+installing+libraries+in+colab&oq=avoid+installing+&gs_lcrp=EgZjaHJvbWUqBggBECMYJzIGCAAQRRg5MgYIARAjGCcyCAgCEAAYFhgeMggIAxAAGBYYHjIMCAQQABgKGA8YFhgeMg0IBRAAGIYDGIAEGIoFMg0IBhAAGIYDGIAEGIoFMg0IBxAAGIYDGIAEGIoFMgoICBAAGIAEGKIEMgoICRAAGIAEGKIE0gEINzE4NGowajeoAgiwAgE&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:48b634c5,vid:ErwLcnvhW4A,st:0)

Guide:
[Link1](https://medium.com/the-ai-forum/instruction-fine-tuning-gemma-2b-on-medical-reasoning-and-convert-the-finetuned-model-into-gguf-844191f8d329)
[Link2](https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887)

In [ ]:
# import os, sys
# from google.colab import drive
# drive.mount('/content/drive') #force_remount=True
# nb_path = '/content/packages'
# os.symlink('/content/drive/My Drive/colab packages', nb_path)
# sys.path.insert(0,nb_path)

Mounted at /content/drive


In [ ]:
# drive.flush_and_unmount()

#Install Libraries

In [ ]:
# !pip install transformers==4.32.0 peft==0.5.0
# !pip install sentencepiece
# !pip install --target=$nb_path accelerate
!pip install accelerate
# !pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install trl

#Load Libraries

In [ ]:
"""
To install a library, run the following command:
!pip install library_name
"""
import warnings
warnings.filterwarnings('ignore') # to avoid warnings
import os

# from transformers import BertTokenizer,  AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0
import torch
from transformers import BitsAndBytesConfig

"""
Pytorch Libraries
"""
# import torch
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

'\nPytorch Libraries\n'

# Quantization using bitsandbytes

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True, llm_int8_enable_fp32_cpu_offload=True)
#llm_int8_enable_fp32_cpu_offload=True

# Load Model and tokenizer

In [ ]:
# Load Models
base_model = "NousResearch/Llama-2-13b-hf"
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
#The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
#The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions.
model = LlamaForCausalLM.from_pretrained(base_model, device_map = "cuda:0",quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

# Load Peft version of model

In [ ]:
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
peft_model_1 = PeftModel.from_pretrained(model, peft_model)
# peft_model.train()

In [ ]:
print(peft_model_1.get_memory_footprint())

7123292160


In [ ]:
peft_model_1 = peft_model_1.eval()

In [ ]:
trainable, total = peft_model_1.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 0 | total: 13025694720 | Percentage: 0.0000%


# On a Small example - Prompt

In [ ]:
# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]


In [ ]:
prompt[0]

"Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .\nAnswer: "

In [ ]:
tokenized_set_1 = tokenizer(prompt, return_tensors='pt', truncation=True,padding=True,max_length=200)

# tokenized_set_1 = tokenizer(formatted_text, return_tensors='pt', padding=True, max_length=512)

In [ ]:
type(tokenized_set_1)

**Alternative 1 - no dataloader**

In [ ]:
res = model.generate(**tokenized_set_1, max_length=100)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer:")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

  neutral</s>
  neutral</s>
  neutral</s>


In [ ]:
res_sentences

["<s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .\nAnswer:  neutral</s>",
 '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .\nAnswer:  neutral</s>',
 '</s></s></s></s></s></s></s></s><s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .\nAnswer

**Alternative 2 - dataloader**

In [ ]:
tokenized_set_1

In [ ]:
type(tokenized_set_1['input_ids'])

torch.Tensor

In [ ]:
from torch.utils.data import DataLoader,SequentialSampler,TensorDataset

input_ids_val = tokenized_set_1['input_ids']
attention_masks_val = tokenized_set_1['attention_mask']
dataset_val = TensorDataset(input_ids_val, attention_masks_val)

[Datacollator](https://medium.com/@sujathamudadla1213/what-is-datacollatorwithpadding-in-hugging-face-transformers-12c2b3b2f612)

In [ ]:
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
batch_size = 3
dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)
for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }
  res = model.generate(**inputs, max_length=512)
  res_sentences = [tokenizer.decode(i) for i in res]
  out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

cuda
 neutral</s>
 negative</s>
 positive</s>


In [ ]:
res_sentences

["<s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .\nAnswer:  neutral</s>",
 '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .\nAnswer:  neutral</s>',
 '</s></s></s></s></s></s></s></s><s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .\nAnswer

# On Complete Dataset - Financial Phrasebank

**[Dataset](https://huggingface.co/datasets/financial_phrasebank)**

In [ ]:
from datasets import load_dataset

fpb_dataset = load_dataset("financial_phrasebank",'sentences_allagree')

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [ ]:
fpb_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2264
    })
})

In [ ]:
fpb_dataset['train']['sentence'][:2]

['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m ."]

### INSTRUCTION FORMAT ISSUE -NOT WORKING REASON NOT CLEAR

In [ ]:
from torch.utils.data import DataLoader,SequentialSampler,TensorDataset

input_ids_val = tokenized_set_1['input_ids'][:3]
attention_masks_val = tokenized_set_1['attention_mask'][:3]
dataset_val = TensorDataset(input_ids_val, attention_masks_val)

In [ ]:
batch_size = 3
dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# model.eval()
print(device)
for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }
  # with torch.no_grad():
  res = model.generate(**inputs, max_length=512)

  res_sentences = [tokenizer.decode(i) for i in res]
  out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

cuda


IndexError: list index out of range

In [ ]:
res_sentences

**Observation**

Even though same instruction format is used to produce the result from the given model under consideration, only garbage is being generated.
Forex:- if the triple quotes are replace by double quotes, model fails to generate the output instead garbage is produced.

Issue is with the instruction format.

In [ ]:
def format_input():
  formatted_text = ['''What is the sentiment of this news? Please choose an answer from {}
  News: {} Answer:'''.format("{negative/neutral/positive}",p)  for p in fpb_dataset['train']['sentence'][:2]]
  return formatted_text

formatted_input = format_input()

inputs = tokenizer(formatted_input, max_length=200, truncation=True,padding=True,return_tensors="pt")
fpb_output = model.generate(**inputs, max_length=100) #num_beams=5, max_length = 256, temperature=1.1
# decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
fpb_res_sentences = [tokenizer.decode(i) for i in fpb_output]
fpb_out_text = [o.split("Answer:")[1] for o in fpb_res_sentences]

# show results
for sentiment in fpb_out_text:
    print(sentiment)

</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>$}}%<unk><unk><unk><unk><unk><unk><unk><unk>
<unk><unk><unk><unk><unk><unk><unk><unk><unk>


# **Fine tuning**

##load dataset

In [ ]:
from datasets import load_dataset

fpb_dataset = load_dataset("financial_phrasebank",'sentences_allagree')

## Format dataset

In [ ]:
fpb_dataset['train']['label']
sentiments = []
for label in fpb_dataset['train']['label']:
  if label == 0:
    sentiments.append("negative")
  elif label == 1:
    sentiments.append("neutral")
  else:
    sentiments.append("positive")

fpb_dataset['train'] = fpb_dataset["train"].add_column("sentiment_label", sentiments)

In [ ]:
fpb_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'sentiment_label'],
        num_rows: 2264
    })
})

In [ ]:
def generate_prompt(data_point):
  sentiment_labels = "positive/negative/neutral"
  text = data_point['sentence']
  # label = data_point['sentiment_label']
  formatted_text = """What is the sentiment of this news? Please choose an answer from {}
  News: {} Sentiment:""".format(sentiment_labels,text)
  return formatted_text

In [ ]:
prompt = [generate_prompt(data_point) for data_point in fpb_dataset["train"]]
fpb_dataset_2 = fpb_dataset["train"].add_column("prompt", prompt)
fpb_dataset_2

Dataset({
    features: ['sentence', 'label', 'sentiment_label', 'prompt'],
    num_rows: 2264
})

In [ ]:
# prompt: add columns to dataset object

fpb_dataset_2 = fpb_dataset_2.add_column("prompt", prompt)


In [ ]:
fpb_dataset_2['prompt'][:3]

['What is the sentiment of this news? Please choose an answer from positive/negative/neutral\n  News: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . Sentiment:neutral',
 "What is the sentiment of this news? Please choose an answer from positive/negative/neutral\n  News: For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m . Sentiment:positive",
 'What is the sentiment of this news? Please choose an answer from positive/negative/neutral\n  News: In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn . Sentiment:positive']

In [ ]:
fpb_dataset_2['sentiment_label'][:3]

['neutral', 'positive', 'positive']

##Find Max sequence length

In [ ]:
max_token_length = [len(fin_tokenizer.encode(example)) for example in fpb_dataset_2['prompt']]

In [ ]:
max(max_token_length)

##Shuffle the dataset.

In [ ]:
fpb_dataset_2 = fpb_dataset_2.shuffle(seed=1234)  # Shuffle dataset here

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

##Tokenize dataset

In [ ]:
fpb_dataset_3 = fpb_dataset_2.map(lambda samples: tokenizer(samples["prompt"], max_length=200, truncation=True,padding=True,return_tensors="pt"), batched=True)
fpb_dataset_3['labels'] = fpb_dataset_3.map(lambda samples: tokenizer(samples["sentiment_label"], max_length=15, truncation=True,padding=True,return_tensors="pt"), batched=True) #remove_columns=['sentence','label','sentiment_label','prompt']

In [ ]:
fpb_dataset_3

Dataset({
    features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 2264
})

## Train-Test Split

In [ ]:
fpb_dataset_4 = fpb_dataset_3.train_test_split(test_size=0.3,stratify_by_column="label",seed=1234)

In [ ]:
fpb_dataset_4

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 1584
    })
    test: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 680
    })
})

In [ ]:
fpb_dataset_5 = fpb_dataset_4['test'].train_test_split(test_size=0.5,stratify_by_column="label",seed=1234)

In [ ]:
fpb_dataset_5

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 340
    })
    test: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 340
    })
})

In [ ]:
import datasets
tokenized_ds = datasets.DatasetDict({"train":fpb_dataset_4['train'],"test":fpb_dataset_5['train'],"val":fpb_dataset_5['test']})

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 1584
    })
    test: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 340
    })
    val: Dataset({
        features: ['sentence', 'label', 'sentiment_label', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 340
    })
})

## Remove unnecessary columns

In [ ]:
# prompt: Remove columns from dataset object

final_tokenized_ds = tokenized_ds.remove_columns(['sentence','label','sentiment_label','prompt'])
final_tokenized_ds


## Load a PeftModel

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)
#
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'o_proj', 'gate_proj', 'base_layer', 'up_proj']


In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

## Trainable Parameters

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

## Preparing tensorboard

In [ ]:
model_checkpoint = "fingpt"
task = "Sentiment analysis"
model_output_dir = f"{model_checkpoint}-finetuned-{task}"
print(model_output_dir) # distilbert-base-uncased-finetuned-imdb

# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_output_dir}'/runs

## Compute Metrics method

In [ ]:
# def model_init():
#     return AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
#                                                               num_labels=3)
import evaluate
metric = evaluate.load("accuracy")

# Function that will be called at the end of each evaluation phase on the whole
# arrays of predictions/labels to produce metrics.
def calculate_metric_on_test_ds(data_test,model,tokenizer):
  codes = model.generate(input_ids = torch.tensor(data_test["input_ids"]).to(device),attention_mask = torch.tensor(data_test["attention_mask"]).to(device))
                           #num_beams=1, max_length = 2, temperature=0.1)
  decoded_codes = [tokenizer.decode(s,skip_special_tokens=True) for s in codes]     #clean_up_tokenization_spaces=True

  out_text = [o.split("Sentiment: ")[1] for o in decoded_codes]
  labels=tokenized_ds['sentiment_label']
# show results
  predictions = []
  for sentiment in out_text:
      # print(sentiment)
      predictions.append(sentiment)
  results =  metric.compute(predictions=predictions, references=labels)
  return results

# Since PyTorch does not provide a training loop, the 🤗 Transformers library
# provides a Trainer API that is optimized for 🤗 Transformers models, with a
# wide range of training options and with built-in features like logging,
# gradient accumulation, and mixed precision.


## Setting training arguments and trainer

In [ ]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'
torch.cuda.empty_cache()


training_arg = transformers.TrainingArguments(
        output_dir=model_output_dir,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        evaluation_strategy="steps",
        num_train_epochs=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        warmup_ratio = 0.03,
        lr_scheduler_type = "constant",
        # max_steps=100,
        eval_steps=50,
        logging_strategy="steps",
        logging_steps=50,
        learning_rate=2e-5,
        logging_steps=50,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="steps",
    )


trainer = Trainer(
    model=model,
    train_dataset=final_tokenized_ds['train'],
    eval_dataset=final_tokenized_ds['val'],
    dataset_text_field="prompt",
    peft_config=lora_config,
    max_seq_length=max(max_token_length),
    args = training_arg,
    compute_metrics=compute_metrics,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
#
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

## Test fine tuned model on test set

In [ ]:
score = calculate_metric_on_test_ds(data_test,trainer.model,tokenizer)